# Deep Residual Neural Network

Here, we'll implement a [ResNet](https://arxiv.org/pdf/1512.03385.pdf). Residual networks allow us to train much deeper networks than were previously practically feasible.

See section 4.2 of the paper for the model architecture on CIFAR-10

Some parts of the code were referenced from https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py  

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Hyper-parameters
NUM_EPOCHS = 80
BATCH_SIZE = 100
LEARNING_RATE = 0.001

In [4]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

In [6]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(
    root='../data/',
    train=True, 
    transform=transform,
    download=True
)

test_dataset = torchvision.datasets.CIFAR10(
    root='../data/',
    train=False, 
    transform=transforms.ToTensor()
)

# Data loader
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

Files already downloaded and verified


In [8]:
def conv3x3(in_channels, out_channels, stride=1):
    """3x3 Convolution with padding"""
    return nn.Conv2d(
        in_channels=in_channels,
        out_channels=out_channels,
        kernel_size=3,
        stride=stride,
        padding=1,
        bias=False
    )

class ResidualBlock(nn.Module):
    """Residual Block"""
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)

        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)

        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [11]:
class ResNet(nn.Module):
    """ResNet"""
    
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16

        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)

        self.avg_pool = nn.AvgPool2d(8)

        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None

        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out

In [12]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [13]:
def update_lr(optimizer, lr):
    """For updating learning rate"""
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [14]:
# Train the model
total_step = len(train_loader)
curr_lr = LEARNING_RATE

for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500], Loss: 1.5421
Epoch [1/80], Step [200/500], Loss: 1.4903
Epoch [1/80], Step [300/500], Loss: 1.3340
Epoch [1/80], Step [400/500], Loss: 1.1831
Epoch [1/80], Step [500/500], Loss: 1.0664
Epoch [2/80], Step [100/500], Loss: 1.2803
Epoch [2/80], Step [200/500], Loss: 1.1210
Epoch [2/80], Step [300/500], Loss: 0.8984
Epoch [2/80], Step [400/500], Loss: 0.9346
Epoch [2/80], Step [500/500], Loss: 1.0011
Epoch [3/80], Step [100/500], Loss: 0.9854
Epoch [3/80], Step [200/500], Loss: 1.0280
Epoch [3/80], Step [300/500], Loss: 0.7722
Epoch [3/80], Step [400/500], Loss: 0.9085
Epoch [3/80], Step [500/500], Loss: 0.8256
Epoch [4/80], Step [100/500], Loss: 0.7830
Epoch [4/80], Step [200/500], Loss: 0.7047
Epoch [4/80], Step [300/500], Loss: 0.7643
Epoch [4/80], Step [400/500], Loss: 0.7004
Epoch [4/80], Step [500/500], Loss: 0.6147
Epoch [5/80], Step [100/500], Loss: 0.7868
Epoch [5/80], Step [200/500], Loss: 0.7090
Epoch [5/80], Step [300/500], Loss: 0.6030
Epoch [5/80

In [17]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 87.99 %


In [16]:
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')